# VancouverPy: Restaurant Success Prediction Model Training

This notebook contains the machine learning pipeline for predicting restaurant success scores based on location and environmental factors.

## Table of Contents
1. [Data Loading and Exploration](#data-loading)
2. [Exploratory Data Analysis](#eda)
3. [Feature Selection and Engineering](#feature-engineering)
4. [Model Training and Evaluation](#model-training)
5. [Model Interpretation](#interpretation)
6. [Prediction and Visualization](#prediction)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from folium.plugins import HeatMap

# Machine Learning imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb

# Visualization settings
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

## 1. Data Loading and Exploration {#data-loading}

In [ ]:
# Load processed data
try:
    df_restaurants = pd.read_csv('../data/processed/restaurants_with_features.csv')
    X = pd.read_csv('../data/processed/model_features.csv')
    feature_names = pd.read_csv('../data/processed/feature_names.csv')['feature'].tolist()
    
    print(f"Dataset loaded successfully!")
    print(f"Number of restaurants: {len(df_restaurants)}")
    print(f"Number of features: {len(feature_names)}")
    
except FileNotFoundError:
    print("Processed data not found. Please run the data collection and processing scripts first.")
    print("Run: python src/01_get_data.py")
    print("Then: python src/02_clean_and_feature_engineer.py")

In [ ]:
# Basic data exploration
print("Dataset Info:")
print(f"Shape: {df_restaurants.shape}")
print(f"\nColumns: {list(df_restaurants.columns)}")
print(f"\nFeature Names: {feature_names}")

# Display first few rows
df_restaurants.head()

In [ ]:
# Check for missing values
missing_values = df_restaurants.isnull().sum()
missing_percentage = (missing_values / len(df_restaurants)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percentage
})

print("Missing Values Summary:")
print(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False))

## 2. Exploratory Data Analysis {#eda}

In [ ]:
# Success Score distribution
if 'success_score' in df_restaurants.columns:
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(df_restaurants['success_score'], bins=30, alpha=0.7, edgecolor='black')
    plt.title('Distribution of Success Scores')
    plt.xlabel('Success Score')
    plt.ylabel('Frequency')
    
    plt.subplot(1, 2, 2)
    plt.boxplot(df_restaurants['success_score'])
    plt.title('Success Score Box Plot')
    plt.ylabel('Success Score')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Success Score Statistics:")
    print(df_restaurants['success_score'].describe())
else:
    print("Success score not found in dataset")

In [ ]:
# Feature correlations
if len(X.columns) > 1:
    plt.figure(figsize=(12, 10))
    
    # Calculate correlation matrix
    corr_matrix = X.corr()
    
    # Create heatmap
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.2f', cbar_kws={"shrink": .8})
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # Find highly correlated features
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.8:
                high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
    
    if high_corr_pairs:
        print("Highly correlated feature pairs (|correlation| > 0.8):")
        for pair in high_corr_pairs:
            print(f"{pair[0]} - {pair[1]}: {pair[2]:.3f}")
    else:
        print("No highly correlated feature pairs found.")

In [ ]:
# Geographic distribution of restaurants
if 'latitude' in df_restaurants.columns and 'longitude' in df_restaurants.columns:
    plt.figure(figsize=(12, 8))
    
    # Create scatter plot with success score as color
    if 'success_score' in df_restaurants.columns:
        scatter = plt.scatter(df_restaurants['longitude'], df_restaurants['latitude'], 
                            c=df_restaurants['success_score'], cmap='viridis', alpha=0.6, s=30)
        plt.colorbar(scatter, label='Success Score')
    else:
        plt.scatter(df_restaurants['longitude'], df_restaurants['latitude'], alpha=0.6, s=30)
    
    plt.title('Geographic Distribution of Restaurants in Vancouver')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"Coordinate ranges:")
    print(f"Latitude: {df_restaurants['latitude'].min():.4f} to {df_restaurants['latitude'].max():.4f}")
    print(f"Longitude: {df_restaurants['longitude'].min():.4f} to {df_restaurants['longitude'].max():.4f}")

## 3. Feature Selection and Engineering {#feature-engineering}

In [ ]:
# Prepare target variable
if 'success_score' in df_restaurants.columns:
    y = df_restaurants['success_score'].copy()
    print(f"Target variable prepared. Shape: {y.shape}")
    print(f"Target statistics: Mean={y.mean():.3f}, Std={y.std():.3f}")
else:
    print("Error: Success score not found. Creating a dummy target for demonstration.")
    y = np.random.normal(0.5, 0.2, len(df_restaurants))
    y = np.clip(y, 0, 1)  # Ensure values are between 0 and 1

# Ensure X and y have same length
min_length = min(len(X), len(y))
X = X.iloc[:min_length].copy()
y = y.iloc[:min_length] if hasattr(y, 'iloc') else y[:min_length]

print(f"Final dataset shape: X={X.shape}, y={len(y)}")

In [ ]:
# Feature importance using Random Forest
if len(X.columns) > 1 and len(y) > 10:
    # Quick feature importance analysis
    rf_temp = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_temp.fit(X, y)
    
    # Create feature importance dataframe
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf_temp.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance, x='importance', y='feature')
    plt.title('Feature Importance (Random Forest)')
    plt.xlabel('Importance Score')
    plt.tight_layout()
    plt.show()
    
    print("Feature Importance Ranking:")
    print(feature_importance)
else:
    print("Insufficient data for feature importance analysis")

## 4. Model Training and Evaluation {#model-training}

In [ ]:
# Split data into training and testing sets
if len(X) > 20:  # Minimum viable dataset size
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"Training set: {X_train.shape}")
    print(f"Testing set: {X_test.shape}")
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print("Data splitting and scaling completed!")
else:
    print(f"Dataset too small for training ({len(X)} samples). Need at least 20 samples.")
    X_train = X_test = y_train = y_test = None
    X_train_scaled = X_test_scaled = None

In [ ]:
# Train multiple models
models = {}
results = {}

if X_train is not None:
    # Define models to train
    model_configs = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42)
    }
    
    # Train and evaluate each model
    for name, model in model_configs.items():
        print(f"Training {name}...")
        
        try:
            # Use scaled data for linear models, original for tree-based
            if 'Linear' in name or 'Ridge' in name:
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_test_scaled)
            else:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
            
            # Calculate metrics
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            
            # Store results
            results[name] = {
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae,
                'R²': r2
            }
            
            models[name] = model
            
            print(f"{name} - R²: {r2:.3f}, RMSE: {rmse:.3f}")
            
        except Exception as e:
            print(f"Error training {name}: {e}")
    
    print("\nModel training completed!")
else:
    print("Skipping model training due to insufficient data.")

In [ ]:
# Compare model performance
if results:
    results_df = pd.DataFrame(results).T
    results_df = results_df.sort_values('R²', ascending=False)
    
    print("Model Performance Comparison:")
    print(results_df)
    
    # Plot model comparison
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    metrics = ['R²', 'RMSE', 'MAE', 'MSE']
    for i, metric in enumerate(metrics):
        ax = axes[i//2, i%2]
        results_df[metric].plot(kind='bar', ax=ax)
        ax.set_title(f'{metric} Comparison')
        ax.set_ylabel(metric)
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Identify best model
    best_model_name = results_df.index[0]
    best_model = models[best_model_name]
    print(f"\nBest performing model: {best_model_name}")
    print(f"R² Score: {results_df.loc[best_model_name, 'R²']:.3f}")
else:
    print("No model results to display.")
    best_model = best_model_name = None

## 5. Model Interpretation {#interpretation}

In [ ]:
# Feature importance for best model
if best_model is not None and hasattr(best_model, 'feature_importances_'):
    # Get feature importance
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Plot top 10 features
    plt.figure(figsize=(10, 8))
    top_features = feature_importance.head(10)
    sns.barplot(data=top_features, x='importance', y='feature')
    plt.title(f'Top 10 Feature Importance - {best_model_name}')
    plt.xlabel('Importance Score')
    plt.tight_layout()
    plt.show()
    
    print(f"Top 5 Most Important Features for {best_model_name}:")
    print(feature_importance.head())

elif best_model is not None and hasattr(best_model, 'coef_'):
    # For linear models, show coefficients
    coefficients = pd.DataFrame({
        'feature': X.columns,
        'coefficient': best_model.coef_
    }).sort_values('coefficient', key=abs, ascending=False)
    
    plt.figure(figsize=(10, 8))
    top_coef = coefficients.head(10)
    sns.barplot(data=top_coef, x='coefficient', y='feature')
    plt.title(f'Top 10 Feature Coefficients - {best_model_name}')
    plt.xlabel('Coefficient Value')
    plt.tight_layout()
    plt.show()
    
    print(f"Top 5 Features by Coefficient Magnitude for {best_model_name}:")
    print(coefficients.head())

In [ ]:
# Prediction vs Actual scatter plot
if best_model is not None and X_test is not None:
    # Make predictions
    if 'Linear' in best_model_name or 'Ridge' in best_model_name:
        y_pred_best = best_model.predict(X_test_scaled)
    else:
        y_pred_best = best_model.predict(X_test)
    
    # Create scatter plot
    plt.figure(figsize=(10, 8))
    plt.scatter(y_test, y_pred_best, alpha=0.6)
    
    # Add perfect prediction line
    min_val = min(min(y_test), min(y_pred_best))
    max_val = max(max(y_test), max(y_pred_best))
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    
    plt.xlabel('Actual Success Score')
    plt.ylabel('Predicted Success Score')
    plt.title(f'Prediction vs Actual - {best_model_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Calculate residuals
    residuals = y_test - y_pred_best
    
    # Residual plot
    plt.figure(figsize=(10, 6))
    plt.scatter(y_pred_best, residuals, alpha=0.6)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicted Success Score')
    plt.ylabel('Residuals')
    plt.title(f'Residual Plot - {best_model_name}')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"Residual Statistics:")
    print(f"Mean: {residuals.mean():.4f}")
    print(f"Std: {residuals.std():.4f}")
    print(f"Mean Absolute Residual: {abs(residuals).mean():.4f}")

## 6. Prediction and Visualization {#prediction}

In [ ]:
# Create prediction function for new locations
def predict_restaurant_success(latitude, longitude, price_level=2, model=best_model, scaler_obj=scaler):
    """
    Predict success score for a new restaurant location
    """
    if model is None:
        return "No trained model available"
    
    # Create feature vector (simplified - would need actual feature engineering)
    # This is a placeholder implementation
    features = np.array([[latitude, longitude, price_level] + [0] * (len(X.columns) - 3)])
    features = features[:, :len(X.columns)]  # Ensure correct number of features
    
    # Scale if needed
    if 'Linear' in best_model_name or 'Ridge' in best_model_name:
        features_scaled = scaler_obj.transform(features)
        prediction = model.predict(features_scaled)[0]
    else:
        prediction = model.predict(features)[0]
    
    return max(0, min(1, prediction))  # Ensure prediction is between 0 and 1

# Example predictions for different locations in Vancouver
if best_model is not None:
    example_locations = [
        (49.2827, -123.1207, "Downtown Vancouver"),
        (49.2606, -123.2460, "Kitsilano"),
        (49.2488, -123.1003, "Mount Pleasant")
    ]
    
    print("Example Success Score Predictions:")
    for lat, lon, name in example_locations:
        pred_score = predict_restaurant_success(lat, lon)
        print(f"{name}: {pred_score:.3f}")
else:
    print("No model available for predictions")

In [ ]:
# Create a simple heat map visualization
if best_model is not None and 'latitude' in df_restaurants.columns:
    # Create a grid of predictions across Vancouver
    lat_min, lat_max = df_restaurants['latitude'].min(), df_restaurants['latitude'].max()
    lon_min, lon_max = df_restaurants['longitude'].min(), df_restaurants['longitude'].max()
    
    # Create grid
    lat_range = np.linspace(lat_min, lat_max, 20)
    lon_range = np.linspace(lon_min, lon_max, 20)
    
    # Generate predictions for grid points
    grid_predictions = []
    for lat in lat_range:
        for lon in lon_range:
            pred = predict_restaurant_success(lat, lon)
            if isinstance(pred, (int, float)):
                grid_predictions.append([lat, lon, pred])
    
    if grid_predictions:
        grid_df = pd.DataFrame(grid_predictions, columns=['latitude', 'longitude', 'predicted_success'])
        
        # Create heat map visualization
        plt.figure(figsize=(12, 10))
        
        # Scatter plot with predictions
        scatter = plt.scatter(grid_df['longitude'], grid_df['latitude'], 
                            c=grid_df['predicted_success'], cmap='RdYlGn', 
                            s=100, alpha=0.7, edgecolors='black', linewidth=0.5)
        
        # Overlay actual restaurant locations
        plt.scatter(df_restaurants['longitude'], df_restaurants['latitude'], 
                   c='blue', s=20, alpha=0.5, label='Existing Restaurants')
        
        plt.colorbar(scatter, label='Predicted Success Score')
        plt.title('Restaurant Success Prediction Heat Map - Vancouver')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        print(f"Heat map created with {len(grid_predictions)} prediction points")
    else:
        print("Could not generate grid predictions")
else:
    print("Heat map visualization not available")

## Summary and Conclusions

This notebook demonstrated the complete machine learning pipeline for predicting restaurant success in Vancouver:

1. **Data Integration**: Combined multiple data sources including business licenses, Yelp reviews, demographics, and transit data
2. **Feature Engineering**: Created meaningful predictors like competitive density, transit accessibility, and affordability mismatch
3. **Model Training**: Tested multiple algorithms and selected the best performer
4. **Model Interpretation**: Analyzed feature importance and model predictions
5. **Visualization**: Created geographic visualizations and prediction heat maps

### Key Findings:
- [To be filled based on actual model results]
- Most important factors for restaurant success
- Geographic patterns in success predictions
- Model performance and limitations

### Next Steps:
1. Collect more comprehensive real-world data
2. Implement advanced feature engineering
3. Explore ensemble methods and deep learning approaches
4. Deploy model as a web application for real-time predictions
5. Validate predictions with actual business outcomes